In [1]:
import pandas as pd
import numpy as np
import itertools
import pickle
import os
import re

# Gather

In [2]:
decks = pd.read_csv('C:/Users/muroc/Documents/MTG/data/decks.csv')

FileNotFoundError: [Errno 2] File b'C:/Users/muroc/Documents/MTG/data/decks.csv' does not exist: b'C:/Users/muroc/Documents/MTG/data/decks.csv'

In [42]:
decks = pd.read_csv('~/Documents/programs/python/MTGA_rarity_analysis/data/decks.csv')

In [43]:
decks_clean = decks.copy()

# Assess

In [5]:
decks_clean.head()

,deck_list,player,url,name
0,"2 Forest, 1 Island, 1 Mountain, 2 Swamp, 3 Bre...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,"4 Shock, 1 Forest, 1 Island, 1 Mountain, 1 Swa...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,"3 Forest, 1 Island, 2 Plains, 1 Swamp, 4 Breed...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,"1 Island, 1 Mountain, 2 Plains, 3 Swamp, 2 Blo...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,"1 Island, 1 Mountain, 1 Plains, 4 Hallowed Fou...",Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW


In [6]:
decks_clean.tail()

,deck_list,player,url,name
446,"6 Island, 4 Plains, 4 Hallowed Fountain, 3 Gla...",truthordare,https://www.mtgtop8.com/event?e=23812&d=365508...,UW_Control
447,"4 Planar Cleansing, 7 Island, 5 Plains, 4 Hall...",Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control
448,"1 Disenchant, 7 Island, 4 Plains, 4 Hallowed F...",zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control
449,"17 Plains, 4 Loyal Pegasus, 4 Faerie Guidemoth...",Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie
450,"2 Disenchant, 17 Plains, 4 Loyal Pegasus, 4 Fa...",Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie


In [10]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


## Assessment
- [done] deck_list column has , for delimiter but names can also have a comma
- [done] cards are in one column
- [done] multiple cards are in one object
- card number and card name are the same object
- [done] null values in `deck_list` column
- rarities not present in the data

# Clean

## Define
- filter for any lists in `deck_list` that have null values in it

## Code

In [11]:
decks_clean.deck_list = decks_clean[decks_clean.astype(str)['deck_list'] != "['nan']"]

## Test

In [12]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


## Define
- make , in `deck_list` into '|' without mistaking commas in names

## Code

In [14]:
decks_clean.deck_list = decks_clean.deck_list.str.replace(', (?=\d)', '|')

## Define
- split `deck_list` by card

## Code

In [15]:
new = decks_clean.deck_list.str.split('|', expand=True)
decks_clean = decks_clean.join(new)

## Test

In [16]:
decks_clean.head()

,deck_list,player,url,name,0,1,2,3,4,5,...,30,31,32,33,34,35,36,37,38,39
0,2 Forest|1 Island|1 Mountain|2 Swamp|3 Breedin...,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2 Forest,1 Island,1 Mountain,2 Swamp,3 Breeding Pool,2 Mystical Dispute,...,None,None,None,None,None,None,None,None,None,None
1,4 Shock|1 Forest|1 Island|1 Mountain|1 Swamp|1...,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4 Shock,1 Forest,1 Island,1 Mountain,1 Swamp,1 Blood Crypt,...,2 Overgrown Tomb,2 Watery Grave,1 Temple of Mystery,"3 Neoform,",None,None,None,None,None,None
2,3 Forest|1 Island|2 Plains|1 Swamp|4 Breeding ...,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3 Forest,1 Island,2 Plains,1 Swamp,4 Breeding Pool,4 Gilded Goose,...,None,None,None,None,None,None,None,None,None,None
3,1 Island|1 Mountain|2 Plains|3 Swamp|2 Blood C...,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1 Island,1 Mountain,2 Plains,3 Swamp,2 Blood Crypt,1 Hallowed Fountain,...,4 Oath of Kaya,4 Guild Globe,"2 Sorcerous Spyglass,",None,None,None,None,None,None,None
4,1 Island|1 Mountain|1 Plains|4 Hallowed Founta...,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1 Island,1 Mountain,1 Plains,4 Hallowed Fountain,3 Fae of Wishes,4 Fires of Invention,...,"4 Teferi, Time Raveler",1 Time Wipe,"1 Ashiok, Dream Render",1 Blast Zone,4 Interplanar Beacon,1 Karn's Bastion,"1 Sorcerous Spyglass,",None,None,None


## Define 
- melt cards into one column

## Code

In [17]:
decks_clean = pd.melt(decks_clean, id_vars=['deck_list', 'player', 'url', 'name'], value_vars=decks_clean.columns[4:45])

In [18]:
decks_clean = decks_clean.rename(columns={'value':'cards'}).drop(columns=['deck_list', 'variable'])

## Test

In [19]:
decks_clean.shape

(18040, 4)

In [20]:
decks_clean.head()

,player,url,name,cards
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2 Forest
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4 Shock
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3 Forest
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1 Island
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1 Island


## Define
- drop rows that have 'None' in them

## Code

In [21]:
decks_clean.cards = decks_clean.cards.astype('str')

In [22]:
decks_clean = decks_clean[decks_clean.cards != 'None']

In [46]:
decks_clean.to_csv('~/Documents/programs/python/MTGA_rarity_analysis/data/decks.csv', index=False)

## Define
- get numbers out of `cards` column

## Code

In [48]:
new = decks_clean.cards.str.split('(?<=\d)\s', expand=True)
decks_clean = decks_clean.join(new).rename(columns={'name':'deck_name', 0:'num', 1:'card_name'}).drop(columns=['cards'])

## Test

In [49]:
decks_clean.head()

,player,url,deck_name,num,card_name
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2,Forest
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4,Shock
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3,Forest
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1,Island
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1,Island


In [50]:
decks_clean.to_csv('~/Documents/programs/python/MTGA_rarity_analysis/data/decks.csv', index=False)